In [2]:
import numpy as np
import os
import scipy.io as sio
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance
from matplotlib import pyplot as plt
#from nilearn import plotting

In [63]:
costmat = calculate_cost(fc1, fc2)
permutations, roichanges = graph_matching(costmat)

In [62]:
def calculate_cost(fc1, fc2):
    """Cost function: the cost of remapping each node in matrix A (fc1) to every other node in matrix B (fc2). 
        Output is fed into graph matching algorithm
        
        fc1 - FC matrix of the form nROIs x nROIs
        fc2 - "" 
        """

    # number of brain regions
    nROIs=fc1.shape[0]

    # initialize empty matrix
    costmat=np.zeros((nROIs,nROIs))

    for x in range(0,nROIs): #x = time point 1.
        a=fc1[x]

        for y in range(0,nROIs): #y = time point 2.
            b=fc2[y]

            # cost to assign node x in fc1 to node y in fc2.
            costmat[x,y]=distance.euclidean(a,b)   
        
    return costmat

def graph_matching(costmat):
    """Runs graph matching w/ the Hungarian algorithm
    
        Returns:

        permutations - vector of integers corresponding to the indices of the nodes to which every node has been assigned. (starts with 0)

        roiswapped - binary vector of size 1xnROIs that contains a 1 if a node has been assigned to a different node (i.e., was swapped)
    
    """
    nROIs=costmat.shape[0]
    rowind, colind=linear_sum_assignment(costmat) #graph matching part.

    permutations = np.array([rowind, colind])
    
    roichanges=np.zeros((nROIs,1))
    for i in range(0,nROIs):
        if colind[i]!=rowind[i]:
            roichanges[i]=1 #indices that are switched

    return permutations, roichanges